In [1]:
include("cell.jl")
include("air.jl")
include("boundary_condition.jl")
#include("wall.jl")

aldm_by_alphac (generic function with 1 method)


# 1. Heat and Mass Transferモジュール

【概要】  
多孔質材料中における熱水分同時移動計算は、
- 熱水分移動の支配方程式
- 材料の熱水分状態量と物性値の関係
- 差分法による数値計算
- Lewis関係など空気（流体）の移動力学
など多様な要素から構成される。  

本モジュールは中でも前進型有限差分法による熱水分移動を記述するためのモジュールである。 


- $q_s$ 熱流（固体熱伝導）[J/m2s]
- $j_v$ 水蒸気流 [kg/m2s]
- $j_l$ 液水流 [kg/m2s]
  


## Appendix.） 事前準備：物性値の平均化方法   
差分法では流量を計算するにあたり、2質点間の物性値が必要となるが、2質点間の物性値は通常各質点における物性値を平均化することで得られる。  
以下では物性値の平均化方法について以下の2通りを示す。

### A.1 調和平均  

2質点間の物性値が大きく異なる際に採用される平均化方法である。  
2質点間の流れが定常と仮定すると、その間の物性値は調和平均を用いて以下のように与えられる。  

熱伝導抵抗：  $R[m^2K/W] =\frac{ dx_{l+1} } { \lambda_{l+1} } + \frac{ dx_{l} }{ \lambda_{l} }  $  

これは熱抵抗と同様の考え方であり、各質点の物性値の抵抗値の和として表すことができる。  

同様に水分に関しても抵抗値の和として計算することで物性値を平均化することができる。  
ここで熱伝導率・水分伝導率を合わせて$\lambda$と表記すると、平均の伝導率は以下のように表すことができる。  

平均の伝導率：  $\lambda =\frac{ dx_{l+1} + dx_{l} }{\frac{ dx_{l+1} } { \lambda_{l+1} } + \frac{ dx_{l} }{ \lambda_{l} } } $  


In [2]:
# 抵抗値の平均化
function sum_resistance( ;val_mns::Float64, val_pls::Float64, len_mns::Float64, len_pls::Float64 );
    #if val_mns <= 0 or val_pls <= 0:
    #    lam = 0.0 
    #else:
    return (len_mns + len_pls) / ( len_mns / val_mns + len_pls / val_pls )
end

sum_resistance (generic function with 1 method)

### A.2 加重平均あるいは算術平均
一方でセルの長さに応じた物性値の加重平均を取るという考え方もある。  
この平均化の仕方は一方の伝導率が他方に比べあまりにも値が小さく、現実にそぐわない場合などに用いられる。

物性値の加重平均：　　$\lambda_{ave}=\frac{\lambda_{l+1} dx_{l+1} + \lambda_{l} dx_{l} }{ dx_{l+1} + dx_{l} }$  


In [3]:
# 加重平均による平均化
function cal_mean_average( ;val_mns::Float64, val_pls::Float64, len_mns::Float64, len_pls::Float64 );
    return ( val_mns * len_mns + val_pls * len_pls ) / ( len_mns + len_pls )
end

# （境界面）貫流値として足し合わせる場合
function cal_transmittance( ;alpha::Float64, lam::Float64, dx2::Float64 )
    return ( 1.0 ) / ( 1.0 / alpha + dx / ( 2.0 * lam ) )
end

cal_transmittance (generic function with 1 method)


# 2. 熱流 


## 2.1 熱伝導（フーリエの法則）  
### 2.1.1 基礎方程式
定義：$\dot q = -\lambda\nabla・T$  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\lambda$：熱伝導率[W/mK]  
$T$：絶対温度[K]  

In [4]:
# $$$$$$ 熱 $$$$$$
# 熱伝導：基礎式
cal_heat_conduction( ;lam::Float64, dtemp::Float64, dx2::Float64 ) = - lam * dtemp / dx2

cal_heat_conduction (generic function with 1 method)

### 2.1.2 差分化方程式
#### その１）調和平均による計算  
定義：$\dot q = -\lambda_{ave.} \frac{T_{l+1}-T_{l}} {dx_{l+1}+dx_{l}}$  
$\lambda_{ave.}$：平均の熱伝導率  
$l, l+1$：ある点のセル・隣接する点のセル  
$dx$：セルの質点からセル境界までの距離

In [5]:
# 差分方程式（質点の位置に注意）
function cal_heat_conduction_diff( ;lam_mns::Float64, lam_pls::Float64, temp_mns::Float64, temp_pls::Float64, dx2_mns::Float64, dx2_pls::Float64 )
    lam = sum_resistance( val_mns = lam_mns, val_pls = lam_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_heat_conduction( lam = lam, dtemp = temp_pls - temp_mns, dx2 = dx2_mns + dx2_pls )
end

cal_heat_conduction_diff (generic function with 1 method)

#### その２）加重平均による計算  

In [6]:
# 加重平均による計算方法
function cal_heat_conduction_diff_meanAve( ;lam_mns::Float64, lam_pls::Float64, temp_mns::Float64, temp_pls::Float64, dx2_mns::Float64, dx2_pls::Float64 )
    lam = cal_mean_average( val_mns = lam_mns, val_pls = lam_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_heat_conduction( lam = lam, dtemp = temp_pls - temp_mns, dx2 = dx2_mns + dx2_pls )
end

cal_heat_conduction_diff_meanAve (generic function with 1 method)

### 2.1.3 構造体cellを用いた式
        
引数（キーワード引数）：  
- cell_mns: mns側に来るCell構造体を入力
- cell_pls: pls側に来るCell構造体を入力

戻り値：
- 実数Float64（熱流量）

※調和平均を採用

In [ ]:
function cal_q( cell_mns::Cell, cell_pls::Cell )
    return cal_heat_conduction_diff( lam_mns = lam(cell_mns), lam_pls= lam(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls) )
end
cal_q(;cell_mns, cell_pls ) = cal_q( cell_mns, cell_pls )

cal_q (generic function with 2 methods)

#### 多次元計算用

In [ ]:
# x軸方向
function cal_qx( cell_mns::Cell, cell_pls::Cell )
    return cal_heat_conduction_diff( lam_mns = lam(cell_mns), lam_pls= lam(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls) )
end
cal_qx(;cell_mns, cell_pls ) = cal_qx( cell_mns, cell_pls )
# y軸方向
function cal_qy( cell_mns::Cell, cell_pls::Cell )
    return cal_heat_conduction_diff( lam_mns = lam(cell_mns), lam_pls= lam(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns = dy(cell_mns) - dy2(cell_mns), dx2_pls = dy2(cell_pls) )
end
cal_qy(;cell_mns, cell_pls ) = cal_qy( cell_mns, cell_pls )
# x軸方向
function cal_qz( cell_mns::Cell, cell_pls::Cell )
    return cal_heat_conduction_diff( lam_mns = lam(cell_mns), lam_pls= lam(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns = dz(cell_mns) - dz2(cell_mns), dx2_pls = dz2(cell_pls) )
end
cal_qy(;cell_mns, cell_pls ) = cal_qy( cell_mns, cell_pls )

cal_qy (generic function with 2 methods)

※cell間の物性値の計算方法として加重平均を採用する場合はこちら

In [ ]:
function cal_q_meanAve( cell_mns::Cell, cell_pls::Cell )
    return cal_heat_conduction_diff_meanAve( lam_mns = lam(cell_mns), lam_pls= lam(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls) )
end
cal_q_meanAve(; cell_mns, cell_pls ) = cal_q_meanAve(cell_mns, cell_pls)

cal_q_meanAve (generic function with 2 methods)

## 2.2 熱伝達・熱貫流  
### 2.2.1 基礎方程式

定義：$\dot q = \alpha(T_{air} - T_{wall})$  
空気に接する壁表面には空気の熱伝達層が存在すると考えた場合、もしくは質点間の熱貫流率として計算した場合の熱流。  
ここでは便宜上、空気と壁表面の熱伝達を想定し、空気から壁表面への熱流を正とした。

$\dot q$：単位時間当たりの熱流量の密度[W/m2]  ]
$\alpha$：熱伝達率・熱貫流率[W/m2K]  
$T_{air}$：空気の絶対温度[K]  
$T_{wall}$：壁表面の絶対温度[K]  

In [10]:
######################
# 熱伝達：基礎式
cal_heat_transfer( ;alpha::Float64, dtemp::Float64 ) = - alpha *  dtemp

cal_heat_transfer (generic function with 1 method)

### 2.2.2 差分化方程式

In [11]:
#cal_heat_transfer( ;alpha::Float64, temp_mns::Float64, temp_pls::Float64 ) = cal_heat_transfer( alpha = alpha, dtemp = temp_pls - temp_mns )
cal_heat_transfer_diff( ;alpha::Float64, temp_mns::Float64, temp_pls::Float64 ) = cal_heat_transfer( alpha = alpha, dtemp = temp_pls - temp_mns )

cal_heat_transfer_diff (generic function with 1 method)

あるいは、質点の位置に応じて下記のような記述方法が出来る。

In [12]:
# 質点が材料内部にある場合（壁は右（pls）側）
function cal_heat_transfer_plswall( ;alpha_mns::Float64, lam_pls::Float64, temp_mns::Float64, temp_pls::Float64, dx2_pls::Float64 );
    alpha = cal_transmittance( alpha = alpha_mns, lam = lam_pls, dx2 = dx2_pls )
    return cal_heat_transfer( alpha = alpha, dtemp = temp_pls - temp_mns )
end

# 質点が材料内部にある場合（壁は左（mns）側）
function cal_heat_transfer_mnswall( ;alpha_pls::Float64, lam_mns::Float64, temp_mns::Float64, temp_pls::Float64, dx2_mns::Float64 );
    alpha = cal_transmittance( alpha = alpha_pls, lam = lam_mns, dx2 = dx2_mns )
    return cal_heat_transfer( alpha = alpha, dtemp = temp_pls - temp_mns )
end

cal_heat_transfer_mnswall (generic function with 1 method)

### 2.2.3 構造体cellを用いた式
        
引数（キーワード引数）：  
- cell_mns: mns側に来るCellあるいはBC_Robin構造体を入力
- cell_pls: pls側に来るCellあるいはBC_Robin構造体を入力

なお、境界面における熱伝達は空気がpls側かmns側のどちらに来るかで計算式が変化する。  
そのためここでは多重ディスパッチを用いることでこの問題を解消した。  
（以下はmns側が空気の場合）

戻り値：
- 実数Float64（熱流量）

※調和平均を採用

In [ ]:
# Cell-BC_Robin間
# mns側が空気
function cal_q( cell_mns::BC_Robin, cell_pls::Cell )
    return cal_heat_transfer_diff( 
        alpha    = alpha(cell_mns), 
        temp_mns = temp(cell_mns.air), 
        temp_pls = temp(cell_pls) ) + cell_mns.q_added
end
cal_qx( cell_mns::BC_Robin, cell_pls::Cell ) = cal_q( cell_mns, cell_pls )
cal_qy( cell_mns::BC_Robin, cell_pls::Cell ) = cal_q( cell_mns, cell_pls )
cal_qz( cell_mns::BC_Robin, cell_pls::Cell ) = cal_q( cell_mns, cell_pls )

# pls側が空気
function cal_q( cell_mns::Cell, cell_pls::BC_Robin )
    return cal_heat_transfer_diff( 
        alpha    = alpha(cell_pls), 
        temp_mns = temp(cell_mns), 
        temp_pls = temp(cell_pls.air) ) - cell_pls.q_added
end
cal_qx( cell_mns::Cell, cell_pls::BC_Robin ) = cal_q( cell_mns, cell_pls )
cal_qy( cell_mns::Cell, cell_pls::BC_Robin ) = cal_q( cell_mns, cell_pls )
cal_qz( cell_mns::Cell, cell_pls::BC_Robin ) = cal_q( cell_mns, cell_pls )

cal_qz (generic function with 3 methods)

In [ ]:
# Cell-第一種境界条件間
cal_q(  cell_mns::Cell, cell_pls::BC_Dirichlet ) = cal_q( cell_mns, cell_pls.cell )
cal_q(  cell_mns::BC_Dirichlet, cell_pls::Cell ) = cal_q( cell_mns.cell, cell_pls )
# Cell-第二種境界条件間
cal_q(  cell_mns::Cell, cell_pls::BC_Neumann ) = cell_pls.q
cal_q(  cell_mns::BC_Neumann, cell_pls::Cell ) = cell_mns.q
# BC_Robin-BC_Robin間
cal_q(  cell_mns::BC_Robin, cell_pls::BC_Robin ) = 0.0
# 第二種境界条件-第二種境界条件間
cal_q(  cell_mns::BC_Neumann, cell_pls::BC_Neumann ) = 0.0
# BC_Robin-第二種境界条件間
cal_q(  cell_mns::BC_Robin, cell_pls::BC_Neumann ) = 0.0
cal_q(  cell_mns::BC_Neumann, cell_pls::BC_Robin ) = 0.0

# 3次元用
# Cell-第一種境界条件間
cal_qx(  cell_mns::Cell, cell_pls::BC_Dirichlet ) = cal_q( cell_mns, cell_pls.cell )
cal_qx(  cell_mns::BC_Dirichlet, cell_pls::Cell ) = cal_q( cell_mns.cell, cell_pls )
cal_qy(  cell_mns::Cell, cell_pls::BC_Dirichlet ) = cal_q( cell_mns, cell_pls.cell )
cal_qy(  cell_mns::BC_Dirichlet, cell_pls::Cell ) = cal_q( cell_mns.cell, cell_pls )
cal_qz(  cell_mns::Cell, cell_pls::BC_Dirichlet ) = cal_q( cell_mns, cell_pls.cell )
cal_qz(  cell_mns::BC_Dirichlet, cell_pls::Cell ) = cal_q( cell_mns.cell, cell_pls )
# Cell-第二種境界条件間
cal_qx(  cell_mns::Cell, cell_pls::BC_Neumann ) = cell_pls.q
cal_qx(  cell_mns::BC_Neumann, cell_pls::Cell ) = cell_mns.q
cal_qy(  cell_mns::Cell, cell_pls::BC_Neumann ) = cell_pls.q
cal_qy(  cell_mns::BC_Neumann, cell_pls::Cell ) = cell_mns.q
cal_qz(  cell_mns::Cell, cell_pls::BC_Neumann ) = cell_pls.q
cal_qz(  cell_mns::BC_Neumann, cell_pls::Cell ) = cell_mns.q
# BC_Robin-BC_Robin間
cal_qx(  cell_mns::BC_Robin, cell_pls::BC_Robin ) = 0.0
cal_qy(  cell_mns::BC_Robin, cell_pls::BC_Robin ) = 0.0
cal_qz(  cell_mns::BC_Robin, cell_pls::BC_Robin ) = 0.0
# 第二種境界条件-第二種境界条件間
cal_qx(  cell_mns::BC_Neumann, cell_pls::BC_Neumann ) = 0.0
cal_qy(  cell_mns::BC_Neumann, cell_pls::BC_Neumann ) = 0.0
cal_qz(  cell_mns::BC_Neumann, cell_pls::BC_Neumann ) = 0.0
# BC_Robin-第二種境界条件間
cal_qx(  cell_mns::BC_Robin, cell_pls::BC_Neumann ) = 0.0
cal_qx(  cell_mns::BC_Neumann, cell_pls::BC_Robin ) = 0.0
cal_qy(  cell_mns::BC_Robin, cell_pls::BC_Neumann ) = 0.0
cal_qy(  cell_mns::BC_Neumann, cell_pls::BC_Robin ) = 0.0
cal_qz(  cell_mns::BC_Robin, cell_pls::BC_Neumann ) = 0.0
cal_qz(  cell_mns::BC_Neumann, cell_pls::BC_Robin ) = 0.0

cal_qz (generic function with 9 methods)

### ※cal_qについて  
cal_qは複数の関数が存在するが、多重ディスパッチで定義されるため、mnsとplsの引数の型に応じて適宜使用する関数が変化する。  
従って、同一の名称として定義しても問題ない。


# 3. 水蒸気移動  


## 3.1 水蒸気流（水蒸気圧勾配・固体内）  
### 3.1.1 基礎方程式
定義：$J_v = -\lambda^{'}_{p}\nabla・P_v$  
$J_v$：気相水分流量[kg/m2s]  
$\lambda^{'}_{p}$：水蒸気圧勾配に関する気相水分伝導率[kg/msPa]  
$P_v$：水蒸気圧[Pa]  

In [15]:
##############################################
# $$$$$$ 湿気（水蒸気） $$$$$$
# 湿気伝導（圧力差流れ）：基礎式
cal_vapour_permeance_pressure( ;dp::Float64, dpv::Float64, dx2::Float64 ) = - dp * dpv / dx2

cal_vapour_permeance_pressure (generic function with 1 method)

### 3.1.2 差分化方程式

In [16]:
# 差分方程式
function cal_vapour_permeance_pressure_diff( ;dp_mns::Float64, dp_pls::Float64, pv_mns::Float64, pv_pls::Float64, dx2_mns::Float64, dx2_pls::Float64 )
    dp = sum_resistance( val_mns = dp_mns, val_pls = dp_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_vapour_permeance_pressure( dp = dp, dpv = pv_pls - pv_mns, dx2 = dx2_mns + dx2_pls )
end

cal_vapour_permeance_pressure_diff (generic function with 1 method)

## 3.2水蒸気流（水分化学ポテンシャル・温度勾配・固体内）  
### 3.2.1 基礎方程式
定義：$J_v = -\lambda ^{'}_{\mu g}\nabla \mu - \lambda ^{'}_{T g}\nabla T$  
$J_v$：気相水分流量[kg/m2s]  
$\mu$：水分化学ポテンシャル[J/kg]  
$T$：絶対温度[K]  
$\lambda ^{'}_{\mu g}$：水分化学ポテンシャル勾配に対する気相水分伝導率[kg/ms(J/kg)]  
$\lambda ^{'}_{T g}$：温度勾配に対する気相水分伝導率[kg/ms(K)]  

In [17]:
# 湿気伝導（水分化学ポテンシャル流れ）
cal_vapour_permeance_potential( ;ldmg::Float64, ldtg::Float64, dmiu::Float64, dtemp::Float64, dx2::Float64 ) = - ( ldmg * dmiu / dx2 + ldtg * dtemp / dx2 )

cal_vapour_permeance_potential (generic function with 1 method)

### 3.2.2 差分化方程式

In [18]:
# 差分方程式
function cal_vapour_permeance_potential_diff( ;ldmg_mns::Float64, ldmg_pls::Float64, ldtg_mns::Float64, ldtg_pls::Float64, miu_mns::Float64, miu_pls::Float64, temp_mns::Float64, temp_pls::Float64, dx2_mns::Float64, dx2_pls::Float64 )
    ldmg = sum_resistance( val_mns = ldmg_mns, val_pls = ldmg_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    ldtg = sum_resistance( val_mns = ldtg_mns, val_pls = ldtg_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_vapour_permeance_potential( ldmg = ldmg, ldtg = ldtg, dmiu = miu_pls - miu_mns, dtemp = temp_pls - temp_mns, dx2 = dx2 = dx2_mns + dx2_pls )
end

# 加重平均による計算方法
function cal_vapour_permeance_potential_diff_meanAve( ;ldmg_mns::Float64, ldmg_pls::Float64, ldtg_mns::Float64, ldtg_pls::Float64, miu_mns::Float64, miu_pls::Float64, temp_mns::Float64, temp_pls::Float64, dx2_mns::Float64, dx2_pls::Float64 )
    ldmg = cal_mean_average( val_mns = ldmg_mns, val_pls = ldmg_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    ldtg = cal_mean_average( val_mns = ldtg_mns, val_pls = ldtg_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_vapour_permeance_potential( ldmg = ldmg, ldtg = ldtg, dmiu = miu_pls - miu_mns, dtemp = temp_pls - temp_mns, dx2 = dx2 = dx2_mns + dx2_pls )
end

cal_vapour_permeance_potential_diff_meanAve (generic function with 1 method)

### 3.2.3 構造体cellを用いた式
        
引数（キーワード引数）：  
- cell_mns: mns側に来るCell構造体を入力
- cell_pls: pls側に来るCell構造体を入力

戻り値：
- 実数Float64（水蒸気流量）

In [ ]:
function cal_jv( cell_mns::Cell, cell_pls::Cell )
    cal_vapour_permeance_potential_diff(
        ldmg_mns = ldmg(cell_mns), ldmg_pls = ldmg(cell_pls), 
        ldtg_mns = ldtg(cell_mns), ldtg_pls = ldtg(cell_pls), 
        miu_mns  = miu(cell_mns),  miu_pls  = miu(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns  = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls) )
end
cal_jv(; cell_mns, cell_pls ) = cal_jv(cell_mns, cell_pls)

cal_jv (generic function with 2 methods)

#### 多次元計算用

In [ ]:
# Cell-Cell間
# x軸方向
function cal_jvx( cell_mns::Cell, cell_pls::Cell )
    cal_vapour_permeance_potential_diff(
        ldmg_mns = ldmg(cell_mns), ldmg_pls = ldmg(cell_pls), 
        ldtg_mns = ldtg(cell_mns), ldtg_pls = ldtg(cell_pls), 
        miu_mns  = miu(cell_mns),  miu_pls  = miu(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns  = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls) )
end
cal_jvx(; cell_mns, cell_pls ) = cal_jvx(cell_mns, cell_pls)
# y軸方向
function cal_jvy( cell_mns::Cell, cell_pls::Cell )
    cal_vapour_permeance_potential_diff(
        ldmg_mns = ldmg(cell_mns), ldmg_pls = ldmg(cell_pls), 
        ldtg_mns = ldtg(cell_mns), ldtg_pls = ldtg(cell_pls), 
        miu_mns  = miu(cell_mns),  miu_pls  = miu(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns  = dy(cell_mns) - dy2(cell_mns), dx2_pls = dy2(cell_pls) )
end
cal_jvy(; cell_mns, cell_pls ) = cal_jvy(cell_mns, cell_pls)
# x軸方向
function cal_jvz( cell_mns::Cell, cell_pls::Cell )
    cal_vapour_permeance_potential_diff(
        ldmg_mns = ldmg(cell_mns), ldmg_pls = ldmg(cell_pls), 
        ldtg_mns = ldtg(cell_mns), ldtg_pls = ldtg(cell_pls), 
        miu_mns  = miu(cell_mns),  miu_pls  = miu(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns  = dz(cell_mns) - dz2(cell_mns), dx2_pls = dz2(cell_pls) )
end
cal_jvz(; cell_mns, cell_pls ) = cal_jvz(cell_mns, cell_pls)

cal_jvz (generic function with 2 methods)

※cell間の物性値の計算方法として加重平均を採用する場合はこちら

In [21]:
function cal_jv_meanAve( cell_mns::Cell, cell_pls::Cell )
    cal_vapour_permeance_potential_diff_meanAve( 
        ldmg_mns = ldmg(cell_mns), ldmg_pls = ldmg(cell_pls), 
        ldtg_mns = ldtg(cell_mns), ldtg_pls = ldtg(cell_pls), 
        miu_mns  = miu(cell_mns),  miu_pls  = miu(cell_pls), 
        temp_mns = temp(cell_mns), temp_pls = temp(cell_pls), 
        dx2_mns  = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls) )
end
cal_jv_meanAve(; cell_mns, cell_pls ) = cal_jv_meanAve( cell_mns, cell_pls)

cal_jv_meanAve (generic function with 2 methods)

## 3.3 湿気伝達・湿気貫流  
### 3.3.1 基礎方程式
定義：$J_v = \alpha^{'}_{m}(P_{v_{air}} - P_{v_{wall}})$  
空気に接する壁表面には空気の湿気伝達層が存在すると考えた場合の水分流（水蒸気流）、もしくは質点間の湿気貫流率として計算した場合の水分流。  
ここでは便宜上、空気と壁表面の水分伝達を想定し、空気から壁表面への水分流を正とした。

$J_v$：単位時間当たりの水分流量[kg/m2s]  
$\alpha^{'}_{m}$：湿気伝達率[kg/m2s(Pa)]  
$P_{v_{air}}$：空気の水蒸気圧[Pa]  
$P_{v_{wall}}$：壁表面の水蒸気圧[Pa]  

In [22]:
######################
# 湿気伝達（圧力差流れ）
cal_vapour_transfer_pressure( ;aldm::Float64, dpv::Float64 ) = - aldm * dpv

cal_vapour_transfer_pressure (generic function with 1 method)

### 3.3.2 差分化方程式

In [23]:
function cal_vapour_transfer_pressure_diff( ;aldm::Float64, pv_mns::Float64, pv_pls::Float64 );
    return cal_vapour_transfer_pressure( aldm = aldm, dpv = pv_pls - pv_mns )
end

cal_vapour_transfer_pressure_diff (generic function with 1 method)

あるいは、質点の位置に応じて下記のような記述方法が出来る。

In [24]:
# 質点が材料内部にある場合（壁は右（pls）側）
function cal_vapour_transfer_pressure_plswall( ;aldm_mns::Float64, dp_pls::Float64, pv_mns::Float64, pv_pls::Float64, dx2_pls::Float64 );
    aldm = cal_transmittance( alpha = aldm_mns, lam = dp_pls, dx2 = dx2_pls )
    return cal_vapour_transfer_pressure( aldm = aldm, dpv = pv_pls - pv_mns )
end

# 質点が材料内部にある場合（壁は左（mns）側）
function cal_vapour_transfer_pressure_mnswall( ;aldm_pls::Float64, dp_mns::Float64, pv_mns::Float64, pv_pls::Float64, dx2_mns::Float64 );
    aldm = cal_transmittance( alpha = aldm_pls, lam = dp_mns, dx2 = dx2_mns )
    return cal_vapour_transfer_pressure( aldm = aldm, dpv = pv_pls - pv_mns )
end

cal_vapour_transfer_pressure_mnswall (generic function with 1 method)

### 3.3.3 構造体cellを用いた式 
        
引数（キーワード引数）：  
- cell_mns: mns側に来るCellあるいはBC_Robin構造体を入力
- cell_pls: pls側に来るCellあるいはBC_Robin構造体を入力

戻り値：
- 実数Float64（水蒸気流量）

In [ ]:
# Cell-BC_Robin間
# mns側が空気
function cal_jv( cell_mns::BC_Robin, cell_pls::Cell )
    cal_vapour_transfer_pressure_diff( aldm = aldm(cell_mns), 
        pv_mns = pv(cell_mns.air), pv_pls = pv(cell_pls) ) + cell_mns.jv_added
end
cal_jvx( cell_mns::BC_Robin, cell_pls::Cell ) = cal_jv( cell_mns, cell_pls )
cal_jvy( cell_mns::BC_Robin, cell_pls::Cell ) = cal_jv( cell_mns, cell_pls )
cal_jvz( cell_mns::BC_Robin, cell_pls::Cell ) = cal_jv( cell_mns, cell_pls )

# pls側が空気
function cal_jv( cell_mns::Cell, cell_pls::BC_Robin )
    cal_vapour_transfer_pressure_diff( aldm = aldm(cell_pls), 
        pv_mns = pv(cell_mns), pv_pls = pv(cell_pls.air) ) - cell_pls.jv_added
end
cal_jvx( cell_mns::Cell, cell_pls::BC_Robin ) = cal_jv( cell_mns, cell_pls )
cal_jvy( cell_mns::Cell, cell_pls::BC_Robin ) = cal_jv( cell_mns, cell_pls )
cal_jvz( cell_mns::Cell, cell_pls::BC_Robin ) = cal_jv( cell_mns, cell_pls )

cal_jvz (generic function with 4 methods)

In [ ]:
# Cell-第一種境界条件間
cal_jv(  cell_mns::Cell, cell_pls::BC_Dirichlet ) = cal_jv( cell_mns, cell_pls.cell )
cal_jv(  cell_mns::BC_Dirichlet, cell_pls::Cell ) = cal_jv( cell_mns.cell, cell_pls )
# Cell-第二種境界条件間
cal_jv(  cell_mns::Cell, cell_pls::BC_Neumann ) = cell_pls.jv
cal_jv(  cell_mns::BC_Neumann, cell_pls::Cell ) = cell_mns.jv
# BC_Robin-BC_Robin間
cal_jv(  cell_mns::BC_Robin, cell_pls::BC_Robin ) = 0.0
# 第二種境界条件-第二種境界条件間
cal_jv(  cell_mns::BC_Neumann, cell_pls::BC_Neumann ) = 0.0
# BC_Robin-第二種境界条件間
cal_jv(  cell_mns::BC_Robin, cell_pls::BC_Neumann ) = 0.0
cal_jv(  cell_mns::BC_Neumann, cell_pls::BC_Robin ) = 0.0

# 3次元用
# Cell-第一種境界条件間
cal_jvx(  cell_mns::Cell, cell_pls::BC_Dirichlet ) = cal_jv( cell_mns, cell_pls.cell )
cal_jvx(  cell_mns::BC_Dirichlet, cell_pls::Cell ) = cal_jv( cell_mns.cell, cell_pls )
cal_jvy(  cell_mns::Cell, cell_pls::BC_Dirichlet ) = cal_jv( cell_mns, cell_pls.cell )
cal_jvy(  cell_mns::BC_Dirichlet, cell_pls::Cell ) = cal_jv( cell_mns.cell, cell_pls )
cal_jvz(  cell_mns::Cell, cell_pls::BC_Dirichlet ) = cal_jv( cell_mns, cell_pls.cell )
cal_jvz(  cell_mns::BC_Dirichlet, cell_pls::Cell ) = cal_jv( cell_mns.cell, cell_pls )
# Cell-第二種境界条件間
cal_jvx(  cell_mns::Cell, cell_pls::BC_Neumann ) = cell_pls.jv
cal_jvx(  cell_mns::BC_Neumann, cell_pls::Cell ) = cell_mns.jv
cal_jvy(  cell_mns::Cell, cell_pls::BC_Neumann ) = cell_pls.jv
cal_jvy(  cell_mns::BC_Neumann, cell_pls::Cell ) = cell_mns.jv
cal_jvz(  cell_mns::Cell, cell_pls::BC_Neumann ) = cell_pls.jv
cal_jvz(  cell_mns::BC_Neumann, cell_pls::Cell ) = cell_mns.jv
# BC_Robin-BC_Robin間
cal_jvx(  cell_mns::BC_Robin, cell_pls::BC_Robin ) = 0.0
cal_jvy(  cell_mns::BC_Robin, cell_pls::BC_Robin ) = 0.0
cal_jvz(  cell_mns::BC_Robin, cell_pls::BC_Robin ) = 0.0
# 第二種境界条件-第二種境界条件間
cal_jvx(  cell_mns::BC_Neumann, cell_pls::BC_Neumann ) = 0.0
cal_jvy(  cell_mns::BC_Neumann, cell_pls::BC_Neumann ) = 0.0
cal_jvz(  cell_mns::BC_Neumann, cell_pls::BC_Neumann ) = 0.0
# BC_Robin-第二種境界条件間
cal_jvx(  cell_mns::BC_Robin, cell_pls::BC_Neumann ) = 0.0
cal_jvx(  cell_mns::BC_Neumann, cell_pls::BC_Robin ) = 0.0
cal_jvy(  cell_mns::BC_Robin, cell_pls::BC_Neumann ) = 0.0
cal_jvy(  cell_mns::BC_Neumann, cell_pls::BC_Robin ) = 0.0
cal_jvz(  cell_mns::BC_Robin, cell_pls::BC_Neumann ) = 0.0
cal_jvz(  cell_mns::BC_Neumann, cell_pls::BC_Robin ) = 0.0

cal_jvz (generic function with 10 methods)

### Appendix.3) 水分化学ポテンシャルを用いた湿気伝達の式  

※気液界面ではあまり使われない。  
（微分の関係で誤差が大きくなるため。）

In [27]:
######################
# 湿気伝達（化学ポテンシャル差流れ）
cal_vapour_transfer_potential( ;aldmg::Float64, aldtg::Float64, dmiu::Float64, dtemp::Float64 ) = - aldmg * dmiu - aldtg * dtemp

function cal_vapour_transfer_potential_diff( ;aldmg::Float64, aldtg::Float64, miu_mns::Float64, miu_pls::Float64, temp_mns::Float64, temp_pls::Float64 );
    return cal_vapour_transfer_potential( aldmg = aldmg, aldtg = aldtg, dmiu = miu_pls - miu_mns, dtemp = temp_pls - temp_mns )
end

# 質点が材料内部にある場合（壁は右（pls）側）
function cal_vapour_transfer_potential_plswall( ;aldmg_mns::Float64, aldtg_mns::Float64, ldmg_pls::Float64, ldtg_pls::Float64, miu_mns::Float64, miu_pls::Float64, temp_mns::Float64, temp_pls::Float64, dx2_pls::Float64 );
    aldmg = cal_transmittance( alpha = aldmg_mns, lam = ldmg_pls, dx2 = dx2_pls )
    aldtg = cal_transmittance( alpha = aldtg_mns, lam = ldtg_pls, dx2 = dx2_pls )
    return cal_vapour_transfer_potential( aldmg = aldmg, aldtg = aldtg, dmiu = miu_pls - miu_mns, dtemp = temp_pls - temp_mns )
end

# 質点が材料内部にある場合（壁は左（mns）側）
function cal_vapour_transfer_potential_mnswall( ;aldmg_pls::Float64, aldtg_pls::Float64, ldmg_mns::Float64, ldtg_mns::Float64, miu_mns::Float64, miu_pls::Float64, temp_mns::Float64, temp_pls::Float64, dx2_mns::Float64 );
    aldmg = cal_transmittance( alpha = aldmg_pls, lam = ldmg_mns, dx2 = dx2_pls )
    aldtg = cal_transmittance( alpha = aldtg_pls, lam = ldtg_mns, dx2 = dx2_pls )
    return cal_vapour_transfer_potential( aldmg = aldmg, aldtg = aldtg, dmiu = miu_pls - miu_mns, dtemp = temp_pls - temp_mns )
end

cal_vapour_transfer_potential_mnswall (generic function with 1 method)


# 4. 液水移動  


## 4.1 液水移動（水分化学ポテンシャル勾配・固体内）  
### 4.1.1 基礎方程式
定義：$J_l = -\lambda^{'}_{\mu_l}(\nabla・\mu - n_x g)$    

$J_l$：液相水分流量[kg/m2s]  
$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に関する液相水分伝導率[kg/ms(J/kg)]  
$\mu$：液水の化学ポテンシャル[J/kg]  
$n_x$：重力加速度に対する応答（重力加速度に対して水平なら1、垂直なら0）

In [28]:
##############################################
# $$$$$$ 水（液水） $$$$$$
# 液水伝導：基礎式
cal_liquid_conduction_potential( ;ldml::Float64, dmiu::Float64, dx2::Float64, nx = 0.0 ) = - ldml * ( dmiu / dx2 - nx * 9.806650 )

cal_liquid_conduction_potential (generic function with 1 method)

### 4.1.2 差分化方程式

In [29]:
# 差分方程式
function cal_liquid_conduction_potential_diff( ;ldml_mns::Float64, ldml_pls::Float64, miu_mns::Float64, miu_pls::Float64, dx2_mns::Float64, dx2_pls::Float64, nx = 0.0 ) # nxは特に指定が無い場合0
    ldml = sum_resistance( val_mns = ldml_mns, val_pls = ldml_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_liquid_conduction_potential( ldml = ldml, dmiu = miu_pls - miu_mns, dx2 = dx2_mns + dx2_pls, nx = nx )
end

# 加重平均による計算方法
function cal_liquid_conduction_potential_diff_meanAve( ;ldml_mns::Float64, ldml_pls::Float64, miu_mns::Float64, miu_pls::Float64, dx2_mns::Float64, dx2_pls::Float64, nx = 0.0 ) # nxは特に指定が無い場合0
    ldml = cal_mean_average( val_mns = ldml_mns, val_pls = ldml_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_liquid_conduction_potential( ldml = ldml, dmiu = miu_pls - miu_mns, dx2 = dx2_mns + dx2_pls, nx = nx )
end

cal_liquid_conduction_potential_diff_meanAve (generic function with 1 method)

### 4.1.3 構造体cellを用いた式
        
引数（キーワード引数）：  
- cell_mns: mns側に来るCell構造体を入力
- cell_pls: pls側に来るCell構造体を入力
- nx: 重力に対する応答方向を表す。水平の場合0、垂直方向の場合1であり、垂直下向きを正とする。  

戻り値：
- 実数Float64（液水流量）

In [30]:
function cal_jl( cell_mns::Cell, cell_pls::Cell, nx = 0.0 )
    cal_liquid_conduction_potential_diff( 
        ldml_mns = ldml(cell_mns), ldml_pls = ldml(cell_pls), 
        miu_mns = miu(cell_mns), miu_pls = miu(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls), nx = nx )
end
cal_jl(; cell_mns, cell_pls, nx = 0.0 ) = cal_jl(cell_mns, cell_pls, nx)

cal_jl (generic function with 3 methods)

#### 多次元計算用

In [31]:
# x軸方向
function cal_jlx( cell_mns::Cell, cell_pls::Cell, nx = 0.0 )
    cal_liquid_conduction_potential_diff( 
        ldml_mns = ldml(cell_mns), ldml_pls = ldml(cell_pls), 
        miu_mns = miu(cell_mns), miu_pls = miu(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls), nx = nx )
end
cal_jlx(; cell_mns, cell_pls, nx = 0.0 ) = cal_jlx(cell_mns, cell_pls, nx)
# y軸方向
function cal_jly( cell_mns::Cell, cell_pls::Cell, nx = 0.0 )
    cal_liquid_conduction_potential_diff( 
        ldml_mns = ldml(cell_mns), ldml_pls = ldml(cell_pls), 
        miu_mns = miu(cell_mns), miu_pls = miu(cell_pls), 
        dx2_mns = dy(cell_mns) - dy2(cell_mns), dx2_pls = dy2(cell_pls), nx = nx )
end
cal_jly(; cell_mns, cell_pls, nx = 0.0 ) = cal_jly(cell_mns, cell_pls, nx)
# x軸方向
function cal_jlz( cell_mns::Cell, cell_pls::Cell, nx = 1.0 )
    cal_liquid_conduction_potential_diff( 
        ldml_mns = ldml(cell_mns), ldml_pls = ldml(cell_pls), 
        miu_mns = miu(cell_mns), miu_pls = miu(cell_pls), 
        dx2_mns = dz(cell_mns) - dz2(cell_mns), dx2_pls = dz2(cell_pls), nx = nx )
end
cal_jlz(; cell_mns, cell_pls, nx = 1.0 ) = cal_jlz(cell_mns, cell_pls, nz)

cal_jlz (generic function with 3 methods)

なお、液水は空気中は移動できないため、

In [ ]:
# 1次元用
# Cell-第一種境界条件間
cal_jl(  cell_mns::Cell, cell_pls::BC_Dirichlet, nx = 0.0 ) = cal_jl( cell_mns, cell_pls.cell, nx )
cal_jl(  cell_mns::BC_Dirichlet, cell_pls::Cell, nx = 0.0 ) = cal_jl( cell_mns.cell, cell_pls, nx )
# Cell-BC_Robin間
cal_jl(  cell_mns::BC_Robin, cell_pls::Cell, nx = 0.0 ) = cell_mns.jl_added
cal_jl(  cell_mns::Cell, cell_pls::BC_Robin, nx = 0.0 ) = - cell_pls.jl_added
# Cell-第二種境界条件間
cal_jl(  cell_mns::Cell, cell_pls::BC_Neumann, nx = 0.0 ) = cell_pls.jl
cal_jl(  cell_mns::BC_Neumann, cell_pls::Cell, nx = 0.0 ) = cell_mns.jl
# BC_Robin-BC_Robin間
cal_jl(  cell_mns::BC_Robin, cell_pls::BC_Robin, nx = 0.0 ) = 0.0
# 第二種境界条件-第二種境界条件間
cal_jl(  cell_mns::BC_Neumann, cell_pls::BC_Neumann, nx = 0.0 ) = 0.0
# BC_Robin-第二種境界条件間
cal_jl(  cell_mns::BC_Robin, cell_pls::BC_Neumann, nx = 0.0 ) = 0.0
cal_jl(  cell_mns::BC_Neumann, cell_pls::BC_Robin, nx = 0.0 ) = 0.0

# 3次元用
# Cell-第一種境界条件間
cal_jlx(  cell_mns::Cell, cell_pls::BC_Dirichlet, nx = 0.0 ) = cal_jl( cell_mns, cell_pls.cell, nx )
cal_jlx(  cell_mns::BC_Dirichlet, cell_pls::Cell, nx = 0.0 ) = cal_jl( cell_mns.cell, cell_pls, nx )
cal_jly(  cell_mns::Cell, cell_pls::BC_Dirichlet, nx = 0.0 ) = cal_jl( cell_mns, cell_pls.cell, nx )
cal_jly(  cell_mns::BC_Dirichlet, cell_pls::Cell, nx = 0.0 ) = cal_jl( cell_mns.cell, cell_pls, nx )
cal_jlz(  cell_mns::Cell, cell_pls::BC_Dirichlet, nx = 0.0 ) = cal_jl( cell_mns, cell_pls.cell, nx )
cal_jlz(  cell_mns::BC_Dirichlet, cell_pls::Cell, nx = 0.0 ) = cal_jl( cell_mns.cell, cell_pls, nx )
# Cell-BC_Robin間
cal_jlx( cell_mns::Cell, cell_pls::BC_Robin, nx = 0.0 ) = - cell_pls.jl_added
cal_jly( cell_mns::Cell, cell_pls::BC_Robin, nx = 0.0 ) = - cell_pls.jl_added
cal_jlz( cell_mns::Cell, cell_pls::BC_Robin, nx = 0.0 ) = - cell_pls.jl_added
cal_jlx( cell_mns::BC_Robin, cell_pls::Cell, nx = 0.0 ) = cell_mns.jl_added
cal_jly( cell_mns::BC_Robin, cell_pls::Cell, nx = 0.0 ) = cell_mns.jl_added
cal_jlz( cell_mns::BC_Robin, cell_pls::Cell, nx = 0.0 ) = cell_mns.jl_added
# Cell-第二種境界条件間
cal_jlx(  cell_mns::Cell, cell_pls::BC_Neumann, nx = 0.0 ) = cell_pls.jl
cal_jlx(  cell_mns::BC_Neumann, cell_pls::Cell, nx = 0.0 ) = cell_mns.jl
cal_jly(  cell_mns::Cell, cell_pls::BC_Neumann, nx = 0.0 ) = cell_pls.jl
cal_jly(  cell_mns::BC_Neumann, cell_pls::Cell, nx = 0.0 ) = cell_mns.jl
cal_jlz(  cell_mns::Cell, cell_pls::BC_Neumann, nx = 0.0 ) = cell_pls.jl
cal_jlz(  cell_mns::BC_Neumann, cell_pls::Cell, nx = 0.0 ) = cell_mns.jl
# BC_Robin-BC_Robin間
cal_jlx(  cell_mns::BC_Robin, cell_pls::BC_Robin, nx = 0.0 ) = 0.0
cal_jly(  cell_mns::BC_Robin, cell_pls::BC_Robin, nx = 0.0 ) = 0.0
cal_jlz(  cell_mns::BC_Robin, cell_pls::BC_Robin, nx = 0.0 ) = 0.0
# 第二種境界条件-第二種境界条件間
cal_jlx(  cell_mns::BC_Neumann, cell_pls::BC_Neumann, nx = 0.0 ) = 0.0
cal_jly(  cell_mns::BC_Neumann, cell_pls::BC_Neumann, nx = 0.0 ) = 0.0
cal_jlz(  cell_mns::BC_Neumann, cell_pls::BC_Neumann, nx = 0.0 ) = 0.0
# BC_Robin-第二種境界条件間
cal_jlx(  cell_mns::BC_Robin, cell_pls::BC_Neumann, nx = 0.0 ) = 0.0
cal_jlx(  cell_mns::BC_Neumann, cell_pls::BC_Robin, nx = 0.0 ) = 0.0
cal_jly(  cell_mns::BC_Robin, cell_pls::BC_Neumann, nx = 0.0 ) = 0.0
cal_jly(  cell_mns::BC_Neumann, cell_pls::BC_Robin, nx = 0.0 ) = 0.0
cal_jlz(  cell_mns::BC_Robin, cell_pls::BC_Neumann, nx = 0.0 ) = 0.0
cal_jlz(  cell_mns::BC_Neumann, cell_pls::BC_Robin, nx = 0.0 ) = 0.0

cal_jlz (generic function with 19 methods)

※cell間の物性値の計算方法として加重平均を採用する場合はこちら

In [33]:
function cal_jl_meanAve( cell_mns::Cell, cell_pls::Cell, nx = 0.0 )
    cal_liquid_conduction_potential_diff_meanAve( 
        ldml_mns = ldml(cell_mns), ldml_pls = ldml(cell_pls), 
        miu_mns = miu(cell_mns), miu_pls = miu(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls), nx = nx )
end
cal_jl_meanAve(; cell_mns::Cell, cell_pls::Cell, nx = 0.0 ) = cal_jl_meanAve( cell_mns, cell_pls, nx)

cal_jl_meanAve (generic function with 3 methods)

# ５. 溶液移動  

## 5.1 溶液移動（圧力勾配・固体内・体積流量）  
### 5.1.1 基礎方程式
定義： $J_{vol} = - \frac{K}{\rho_{sw} g} (\nabla・p_c - n_z \rho_{sw} g)$ 

$J_{vol}$：溶液体積流量[m3/m2s]  
$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に関する液相水分伝導率[kg/ms(J/kg)]  
$\mu$：液水の化学ポテンシャル[J/kg]  
$n_x$：重力加速度に対する応答（重力加速度に対して水平なら1、垂直なら0）

In [35]:
##############################################
# $$$$$$ 溶液 $$$$$$
# 溶液伝導：基礎式
cal_solusion_conduction( ;dsw::Float64, dplc::Float64, dx2::Float64, rowsw::Float64, nx = 0.0 ) = - dsw * ( dplc / dx2 - nx * rowsw * 9.806650 )

cal_solusion_conduction (generic function with 1 method)

### 5.1.2 差分化方程式

In [36]:
# 差分方程式
function cal_solusion_conduction_diff( ;dsw_mns::Float64, dsw_pls::Float64, plc_mns::Float64, plc_pls::Float64, dx2_mns::Float64, dx2_pls::Float64, rowsw::Float64, nx = 0.0 ) # nxは特に指定が無い場合0
    dsw = sum_resistance( val_mns = dsw_mns, val_pls = dsw_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_solusion_conduction( dsw = dsw, dplc = plc_pls - plc_mns, dx2 = dx2_mns + dx2_pls, rowsw = rowsw, nx = nx )
end

# 加重平均による計算方法
function cal_solusion_conduction_diff_meanAve( ;dsw_mns::Float64, dsw_pls::Float64, plc_mns::Float64, plc_pls::Float64, dx2_mns::Float64, dx2_pls::Float64, rowsw::Float64, nx = 0.0 ) # nxは特に指定が無い場合0
    dsw = cal_mean_average( val_mns = dsw_mns, val_pls = dsw_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_solusion_conduction( dsw = dsw, dplc = plc_pls - plc_mns, dx2 = dx2_mns + dx2_pls, rowsw = rowsw, nx = nx )
end

cal_solusion_conduction_diff_meanAve (generic function with 1 method)

### 5.1.3 構造体cellを用いた式
        
引数（キーワード引数）：  
- cell_mns: mns側に来るCell構造体を入力
- cell_pls: pls側に来るCell構造体を入力
- nx: 重力に対する応答方向を表す。水平の場合0、垂直方向の場合1であり、垂直下向きを正とする。  

戻り値：
- 実数Float64（溶液体積流量）

In [37]:
function cal_Jvol( cell_mns::Cell, cell_pls::Cell, nx = 0.0 )
    cal_solusion_conduction_diff( 
        dsw_mns = dsw(cell_mns), 
        dsw_pls = dsw(cell_pls), 　#粘性による移動係数の変化もここで加える
        plc_mns = plc(cell_mns), plc_pls = plc(cell_pls), 
        dx2_mns = dx2(cell_mns), dx2_pls = dx2(cell_pls), rowsw = (rowsw(cell_mns) + rowsw(cell_pls)) / 2, nx = nx )　
end
cal_Jvol(; cell_mns, cell_pls, nx = 0.0 ) = cal_Jvol(cell_mns, cell_pls, nx)

cal_Jvol (generic function with 3 methods)

なお、溶液は空気中は移動できないため、

In [38]:
cal_Jvol( cell_mns::Cell, cell_pls::BC_Robin,  nx = 0.0 ) = 0.0
cal_Jvol( cell_mns::BC_Robin,  cell_pls::Cell, nx = 0.0 ) = 0.0

cal_Jvol (generic function with 7 methods)

※cell間の物性値の計算方法として加重平均を採用する場合はこちら

In [39]:
function cal_Jvol_meanAve( cell_mns::Cell, cell_pls::Cell, nx = 0.0 )
    cal_solusion_conduction_diff_meanAve( 
        dsw_mns = dsw(cell_mns), 
        dsw_pls = dsw(cell_pls), 　#粘性による移動係数の変化もここで加える
        plc_mns = plc(cell_mns), plc_pls = plc(cell_pls), 
        dx2_mns = dx2(cell_mns), dx2_pls = dx2(cell_pls), rowsw = (rowsw(cell_mns) + rowsw(cell_pls)) / 2, nx = nx )
end
cal_Jvol_meanAve(; cell_mns::Cell, cell_pls::Cell, nx = 0.0 ) = cal_Jvol_meanAve( cell_mns, cell_pls, nx)

cal_Jvol_meanAve (generic function with 3 methods)

## 5.2 水の移動（固体内・モル流量）  
### 5.2.1 基礎方程式

定義：$j_w = {\frac{1 - (1-{\sigma}) \bar v_s C_s}{\bar v_w}} J_{vol} + \frac{\bar v_s}{\bar v_w} D_s \nabla・C_s$

$j_w$：水のモル流量[mol/m2s]  
$\sigma$：反射係数  
$\bar v_s$：塩の部分モル体積[m3/mol]  
$\bar v_w$：水の部分モル体積[m3/mol]  
$C_s$：塩濃度[mol/m3]  
$D_s$：塩の拡散係数[m2/s]

In [40]:
##############################################
# 水の移動：基礎式
function cal_water_conduction( ;Jvol::Float64, ds::Float64, dcs::Float64, dx2::Float64, cs::Float64, vs::Float64, vw::Float64, sigma::Float64 )
    jw_adv = (1 - (1 - sigma) * vs * cs) / vw * Jvol
    jw_diff = vs / vw * ds * dcs / dx2
    jw = jw_adv + jw_diff
    return jw, jw_adv, jw_diff
end                

cal_water_conduction (generic function with 1 method)

### 5.2.2 差分化方程式

In [41]:
# 差分方程式
function cal_water_conduction_diff( ;Jvol::Float64, ds_mns::Float64, ds_pls::Float64, cs_mns::Float64, cs_pls::Float64, 
                                                dx2_mns::Float64, dx2_pls::Float64, cs::Float64, vs::Float64, vw::Float64, sigma::Float64 )
    ds = sum_resistance( val_mns = ds_mns, val_pls = ds_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_water_conduction( Jvol = Jvol, ds = ds, dcs = cs_pls - cs_mns, dx2 = dx2_mns + dx2_pls, cs = cs, vs = vs, vw = vw, sigma = sigma )
end

# 加重平均による計算方法
function cal_water_conduction_diff_meanAve( ;Jvol::Float64, ds_mns::Float64, ds_pls::Float64, cs_mns::Float64, cs_pls::Float64, 
                                                   dx2_mns::Float64, dx2_pls::Float64, cs::Float64, vs::Float64, vw::Float64, sigma::Float64 ) 
    ds = cal_mean_average( val_mns = ds_mns, val_pls = ds_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_water_conduction( Jvol = Jvol, ds = ds, dcs = cs_pls - cs_mns, dx2 = dx2_mns + dx2_pls, cs = cs, vs = vs, vw = vw, sigma = sigma )
end

cal_water_conduction_diff_meanAve (generic function with 1 method)

### 5.2.3 構造体cellを用いた式
        
引数（キーワード引数）：  
- cell_mns: mns側に来るCell構造体を入力
- cell_pls: pls側に来るCell構造体を入力  

戻り値：
- 実数Float64（水のモル流量）

In [42]:
function cal_jw( cell_mns::Cell, cell_pls::Cell, Jvol )
    
    if Jvol > 0.0; cs_in = cs_vol(cell_mns)
        else; cs_in = cs_vol(cell_pls)
    end
        
    cal_water_conduction_diff( 
        Jvol = Jvol,
        ds_mns = ds(cell_mns), ds_pls = ds(cell_pls), 
        cs_mns = cs_vol(cell_mns), cs_pls = cs_vol(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls), 
        cs = cs_in, vs = (vs(cell_mns) + vs(cell_pls)) / 2, vw = (vw(cell_mns) + vw(cell_pls)) / 2, sigma = sigma(cell_mns) )
end
cal_jw(; cell_mns, cell_pls, Jvol ) = cal_jw( cell_mns, cell_pls, Jvol )

cal_jw (generic function with 2 methods)

なお、溶液は空気中は移動できないため、

In [43]:
cal_jw( cell_mns::Cell, cell_pls::BC_Robin,  Jvol ) = 0.0
cal_jw( cell_mns::BC_Robin,  cell_pls::Cell, Jvol ) = 0.0

cal_jw (generic function with 4 methods)

※cell間の物性値の計算方法として加重平均を採用する場合はこちら

In [44]:
function cal_jw_meanAve( cell_mns::Cell, cell_pls::Cell, Jvol )
    
    if Jvol > 0.0; cs_in = cs_vol(cell_mns)
        else; cs_in = cs_vol(cell_pls)
    end
    
    cal_water_conduction_diff_meanAve( 
        Jvol = Jvol,
        ds_mns = ds(cell_mns), ds_pls = ds(cell_pls), 
        cs_mns = cs_vol(cell_mns), cs_pls = cs_vol(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls), 
        cs = cs_in, vs = (vs(cell_mns) + vs(cell_pls)) / 2, vw = (vw(cell_mns) + vw(cell_pls)) / 2, sigma = sigma(cell_mns) )
end
cal_jw_meanAve(; cell_mns, cell_pls, Jvol ) = cal_jw_meanAve(cell_mns, cell_pls, Jvol )

cal_jw_meanAve (generic function with 2 methods)

## 5.3 塩の移動（固体内・モル流量）  
### 5.3.1 基礎方程式

定義：$j_s = (1-{\sigma}) C_s J_{vol} - D_s \nabla・C_s$

$j_s$：塩のモル流量[mol/m2s]  
$\sigma$：反射係数   
$C_s$：塩濃度[mol/m3]  
$D_s$：塩の拡散係数[m2/s]

In [45]:
##############################################
# 塩の移動：基礎式
function cal_salt_conduction( ;Jvol::Float64, ds::Float64, dcs::Float64, dx2::Float64, cs::Float64, sigma::Float64 )
    # 高取修正
    js_adv = (1 - sigma) * cs * Jvol
    js_diff= - ds * dcs / dx2
    js = js_adv + js_diff
    return js, js_adv, js_diff
end      

cal_salt_conduction (generic function with 1 method)

In [46]:
a, b, c = cal_salt_conduction( Jvol = -10.0e-2, ds = 0.001, dcs = 0.2 , dx2 = 0.1, cs = 1.0 , sigma = 0.0)

(-0.10200000000000001, -0.1, -0.002)

### 5.3.2 差分化方程式

In [47]:
# 差分方程式
function cal_salt_conduction_diff( ;Jvol::Float64, ds_mns::Float64, ds_pls::Float64, cs_mns::Float64, cs_pls::Float64, 
                                                dx2_mns::Float64, dx2_pls::Float64, cs::Float64, sigma::Float64 )
    ds = sum_resistance( val_mns = ds_mns, val_pls = ds_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_salt_conduction( Jvol = Jvol, ds = ds, dcs = cs_pls - cs_mns, dx2 = dx2_mns + dx2_pls, cs = cs, sigma = sigma )
end

# 加重平均による計算方法
function cal_salt_conduction_diff_meanAve( ;Jvol::Float64, ds_mns::Float64, ds_pls::Float64, cs_mns::Float64, cs_pls::Float64, 
                                                   dx2_mns::Float64, dx2_pls::Float64, cs::Float64, sigma::Float64 ) 
    ds = cal_mean_average( val_mns = ds_mns, val_pls = ds_pls, len_mns = dx2_mns, len_pls = dx2_pls )
    return cal_salt_conduction( Jvol = Jvol, ds = ds, dcs = cs_pls - cs_mns, dx2 = dx2_mns + dx2_pls, cs = cs, sigma = sigma )
end

cal_salt_conduction_diff_meanAve (generic function with 1 method)

### 5.3.3 構造体cellを用いた式
        
引数（キーワード引数）：  
- cell_mns: mns側に来るCell構造体を入力
- cell_pls: pls側に来るCell構造体を入力  

戻り値：
- 実数Float64（塩のモル流量）

In [48]:
function cal_js( cell_mns::Cell, cell_pls::Cell, Jvol )
    
    if Jvol > 0.0; cs_in = cs_vol(cell_mns)
        else; cs_in = cs_vol(cell_pls)
    end
    
    cal_salt_conduction_diff( 
        Jvol = Jvol,
        ds_mns = ds(cell_mns), ds_pls = ds(cell_pls), 
        cs_mns = cs_vol(cell_mns), cs_pls = cs_vol(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls), cs = cs_in, sigma = sigma(cell_mns) )
end
cal_js(; cell_mns, cell_pls, Jvol ) = cal_js( cell_mns, cell_pls, Jvol )

cal_js (generic function with 2 methods)

なお、溶液は空気中は移動できないため、

In [49]:
cal_js( cell_mns::Cell, cell_pls::BC_Robin,  Jvol ) = 0.0
cal_js( cell_mns::BC_Robin,  cell_pls::Cell, Jvol ) = 0.0

cal_js (generic function with 4 methods)

※cell間の物性値の計算方法として加重平均を採用する場合はこちら

In [50]:
function cal_js_meanAve( cell_mns::Cell, cell_pls::Cell, Jvol )
    
    if Jvol > 0.0; cs_in = cs_vol(cell_mns)
        else; cs_in = cs_vol(cell_pls)
    end
    
    cal_salt_conduction_diff_meanAve( 
        Jvol = Jvol,
        ds_mns = ds(cell_mns), ds_pls = ds(cell_pls), 
        cs_mns = cs_vol(cell_mns), cs_pls = cs_vol(cell_pls), 
        dx2_mns = dx(cell_mns) - dx2(cell_mns), dx2_pls = dx2(cell_pls), cs = cs_in, sigma = sigma(cell_mns) )
end
cal_js_meanAve(; cell_mns, cell_pls, Jvol ) = cal_js_meanAve(cell_mns, cell_pls, Jvol )

cal_js_meanAve (generic function with 2 methods)

# 6. 常微分方程式の解を用いた熱水分収支式  
※ODE：Ordinary Differential Equation（常微分方程式） 

## 6.1 熱収支式
### 6.1.1 基礎方程式  

熱収支式はその一般系として以下のように記述することができる。  

$$ A_1 \frac{\partial T_{(x,y,z)}}{\partial t} = c_1 T_{(x,y,z)} + c_2 $$ 

このとき、隣接する点の温度・湿度を一定値とみなせば、常微分方程式として解くことができる。  
常微分方程式の解は以下のように与えられる。  

$$ T_{(x,y,z)}^{t+\Delta t} = \Bigl( T_{(x,y,z)}^t + \frac{c_2}{c_1} \Bigl) exp \Bigl( \frac{c_1}{A_1}\Delta t \Bigl) - \frac{c_2}{c_1} $$ 

なお、この解は多孔質材料中、空気中どちらにでも適用可能である。

In [51]:
function cal_newtemp_by_ODE(temp::Float64, A1::Float64, c1::Float64, c2::Float64, dt::Float64 )
    return ( temp + c2 / c1 ) * exp( c1/A1*dt ) - c2 / c1
end
cal_newtemp_by_ODE(;temp::Float64, A1::Float64, c1::Float64, c2::Float64, dt::Float64 ) = cal_newtemp_by_ODE(temp, A1, c1, c2, dt )

cal_newtemp_by_ODE (generic function with 6 methods)

### 6.1.2 各種係数の与え方

#### 6.1.2.1 係数$A_1$の与え方

係数$A_1$は以下のように与えられる。  

##### a) 多孔質材料中

$$ A_1 = c\rho V$$  

In [52]:
cal_A1(c::Float64, rho::Float64, V::Float64) = c * rho * V
cal_A1(cell::Cell) = crow(cell) * vol(cell)

cal_A1 (generic function with 2 methods)

##### b) 空気中  


In [53]:
cal_A1(cell::BC_Robin) = (1005.0 + 1846.0 * ah(cell) ) * ( 353.25/temp(cell) ) * vol(cell)

cal_A1 (generic function with 3 methods)

#### 6.1.2.2 $c_1$の与え方
$c_1$は以下のように与えられる。

##### a) 多孔質材料中

$$ c_{1} = - \sum_{l}^{x,y,z} \sum_{k=0}^{1} \Bigl ( \frac{\lambda_{(l-k)}}{dl2_{(l-k)}} + r \frac{\lambda_{T_g(l-k)}^{'}}{dl2_{(l-k)}} \Bigl ) $$  



$$ c_1 = \sum_{i = x,y,z} \Bigl ( c_{1(i)}^{+} + c_{1(i)}^{-} \Bigl ) $$  

ただし、
$$ c_{1(x)}^{-} = - \Bigl ( \frac{\lambda_{(x-1)}}{dx2_{(x-1)}} + r \frac{\lambda_{Tg(x-1)}^{'}}{dx2_{(x-1)}} \Bigl ) $$  
$$ c_{1(x)}^{+} = - \Bigl ( \frac{\lambda_{(x)}}{dx2_{(x)}} + r \frac{\lambda_{Tg(x)}^{'}}{dx2_{(x)}} \Bigl ) $$  

In [54]:
function cal_c1x(cell_mns::Cell, cell::Cell, cell_pls::Cell)
    return cal_c1x_mns(cell, cell_mns) + cal_c1x_pls(cell, cell_pls)
end

function cal_c1x_mns(cell::Cell, cell_mns::Cell)
    r = ( 597.5 - 0.559 * ( temp(cell) - 273.15 ) ) * 4.18605E+3
    dlam_mns  = 1.0 / ( dlamx_mns(cell) + dlamx_pls(cell_mns)  )
    dldtg_mns = 1.0 / ( dldtgx_mns(cell) + dldtgx_pls(cell_mns)  )
    return - ( dlam_mns + r * dldtg_mns )
end

function cal_c1x_pls(cell::Cell, cell_pls::Cell)
    r = ( 597.5 - 0.559 * ( temp(cell) - 273.15 ) ) * 4.18605E+3
    dlam_pls  = 1.0 / ( dlamx_pls(cell) + dlamx_mns(cell_pls)  )
    dldtg_pls = 1.0 / ( dldtgx_pls(cell) + dldtgx_mns(cell_pls)  )
    return - ( dlam_pls + r * dldtg_pls )
end

cal_c1x_pls (generic function with 2 methods)

##### b) 空気境界

$$ c_{1(x)}^{-} = - ( \alpha + r \alpha_{T}^{'} ) $$  
$$ c_{1(x)}^{+} = - ( \alpha + r \alpha_{T}^{'} ) $$  

In [55]:
cal_c1x(cell_mns::BC_Robin, cell::Cell, cell_pls::Cell)     = cal_c1x_mns(cell, cell_mns) + cal_c1x_pls(cell, cell_pls)
cal_c1x(cell_mns::Cell,     cell::Cell, cell_pls::BC_Robin) = cal_c1x_mns(cell, cell_mns) + cal_c1x_pls(cell, cell_pls)
cal_c1x(cell_mns::BC_Robin, cell::Cell, cell_pls::BC_Robin) = cal_c1x_mns(cell, cell_mns) + cal_c1x_pls(cell, cell_pls)

function cal_c1x_mns(cell::Cell, cell_mns::BC_Robin)
    r = ( 597.5 - 0.559 * ( temp(cell) - 273.15 ) ) * 4.18605E+3
    return - ( alpha(cell_mns) + r * aldt(cell_mns) )
end

function cal_c1x_pls(cell::Cell, cell_pls::BC_Robin)
    r = ( 597.5 - 0.559 * ( temp(cell) - 273.15 ) ) * 4.18605E+3
    return - ( alpha(cell_pls) + r * aldt(cell_pls) )
end

cal_c1x_pls (generic function with 2 methods)

##### c) 空気中  

$$ c_1 = -\Sigma_{k=1}^{m}S_k \alpha -\Sigma_{j=1}^{n} Q_{ij} c_a \rho $$

In [56]:
# メイン
function cal_c1(cell::BC_Robin, cell_trans::Array{Cell,1}, cell_vent::Array{BC_Robin,1}, Q_vent::Array{Float64,1})
    return cal_c1_trans(cell, cell_trans) + cal_c1_vent( cell, cell_vent, Q_vent)
end

# 伝達成分
function cal_c1_trans(cell::BC_Robin, cell_else::Array{Cell,1})
    return - sum( [ alpha(cell) for i = 1:length(cell_else) ] )
end

# 換気成分
function cal_c1_vent( cell::BC_Robin, cell_else::Array{BC_Robin,1}, Q_vent::Array{Float64,1})
    return - sum( [ Q_vent[i] * (1005.0 + 1846.0 * ah(cell)) * (353.25/temp(cell)) for i = 1:length(cell_else) ] )
end

cal_c1_vent (generic function with 1 method)

#### 6.1.2.3 $c_2$の与え方
$c_2$は以下のように与えられる。

##### a) 多孔質材料中

$$ c_2 = \sum_{i = x,y,z} \Bigl ( c_{2(i)}^{+} + c_{2(i)}^{-} \Bigl ) $$  
ただし、
$$ c_{2(x)}^{-} = \Bigl ( \frac{\lambda_{(x-1)}}{dx2_{(x-1)}} + r \frac{\lambda_{Tg(x-1)}^{'}}{dx2_{(x-1)}} \Bigl ) T_{(x-1)} 
- r \frac{\lambda_{\mu g(x-1)}^{'}}{dx2_{(x-1)}} \Bigl ( \mu_{(x)} - \mu_{(x-1)} \Bigl )$$  
$$ c_{2(x)}^{+} = \Bigl ( \frac{\lambda_{(x)}}{dx2_{(x)}} + r \frac{\lambda_{Tg(x)}^{'}}{dx2_{(x)}} \Bigl ) T_{(x+1)} 
- r \frac{\lambda_{\mu g(x)}^{'}}{dx2_{(x)}} \Bigl ( \mu_{(x)} - \mu_{(x+1)} \Bigl )$$  

In [57]:
function cal_c2x(cell_mns::Cell, cell::Cell, cell_pls::Cell)
    return cal_c2x_mns(cell, cell_mns) + cal_c2x_pls(cell, cell_pls)
end

function cal_c2x_mns(cell::Cell, cell_mns::Cell)
    r = ( 597.5 - 0.559 * ( temp(cell) - 273.15 ) ) * 4.18605E+3
    dlam_mns  = 1.0 / ( dlamx_mns(cell) + dlamx_pls(cell_mns)  )
    dldtg_mns = 1.0 / ( dldtgx_mns(cell) + dldtgx_pls(cell_mns)  )
    dldmg_mns = 1.0 / ( dldmgx_mns(cell) + dldmgx_pls(cell_mns)  )
    return ( dlam_mns + r * dldtg_mns ) * temp(cell_mns) - r * dldmg_mns * ( miu(cell) - miu(cell_mns) )
end

function cal_c2x_pls(cell::Cell, cell_pls::Cell)
    r = ( 597.5 - 0.559 * ( temp(cell) - 273.15 ) ) * 4.18605E+3
    dlam_pls  = 1.0 / ( dlamx_pls(cell) + dlamx_mns(cell_pls)  )
    dldtg_pls = 1.0 / ( dldtgx_pls(cell) + dldtgx_mns(cell_pls)  )
    dldmg_pls = 1.0 / ( dldmgx_pls(cell) + dldmgx_mns(cell_pls)  )
    return ( dlam_pls + r * dldtg_pls ) * temp(cell_pls) - r * dldmg_pls * ( miu(cell) - miu(cell_pls) )
end

cal_c2x_pls (generic function with 2 methods)

##### b)空気境界

$$ c_{2(x)}^{-} = ( \alpha + r \alpha_{T}^{'} ) T_{(x-1)} - r \alpha_{\mu}^{'} \Bigl ( \mu_{(x)} - \mu_{(x-1)} \Bigl )$$  
$$ c_{2(x)}^{+} = ( \alpha + r \alpha_{T}^{'} ) T_{(x+1)} - r \alpha_{\mu}^{'} \Bigl ( \mu_{(x)} - \mu_{(x+1)} \Bigl ) $$  

In [58]:
cal_c2x(cell_mns::BC_Robin, cell::Cell, cell_pls::Cell)     = cal_c2x_mns(cell, cell_mns) + cal_c2x_pls(cell, cell_pls)
cal_c2x(cell_mns::Cell,     cell::Cell, cell_pls::BC_Robin) = cal_c2x_mns(cell, cell_mns) + cal_c2x_pls(cell, cell_pls)
cal_c2x(cell_mns::BC_Robin, cell::Cell, cell_pls::BC_Robin) = cal_c2x_mns(cell, cell_mns) + cal_c2x_pls(cell, cell_pls)

function cal_c2x_mns(cell::Cell, cell_mns::BC_Robin)
    r = ( 597.5 - 0.559 * ( temp(cell) - 273.15 ) ) * 4.18605E+3
    return ( alpha(cell_mns) + r * aldt(cell_mns) ) * temp(cell_mns) - r * aldmu(cell_mns) * ( miu(cell) - miu(cell_mns) )
end

function cal_c2x_pls(cell::Cell, cell_pls::BC_Robin)
    r = ( 597.5 - 0.559 * ( temp(cell) - 273.15 ) ) * 4.18605E+3
    return ( alpha(cell_pls) + r * aldt(cell_pls) ) * temp(cell_pls) - r * aldmu(cell_pls) * ( miu(cell) - miu(cell_pls) )
end

cal_c2x_pls (generic function with 2 methods)

##### c) 空気中  

$$ c_2 = \Sigma_{k=1}^{m}S_k \alpha T_k + \Sigma_{j=1}^{n} Q_{ij} c_a \rho T_j + H $$

In [59]:
# メイン
function cal_c2(cell::BC_Robin, cell_trans::Array{Cell,1}, cell_vent::Array{BC_Robin,1}, Q_vent::Array{Float64,1}, Hi::Float64)
    return cal_c2_trans(cell, cell_trans) + cal_c2_vent( cell, cell_vent, Q_vent) + cal_c2_source(Hi)
end

# 伝達成分
function cal_c2_trans(cell::BC_Robin, cell_else::Array{Cell,1})
    return sum( [alpha(cell) * temp(cell_else[i]) for i = 1:length(cell_else) ] )
end

# 換気成分
function cal_c2_vent( cell::BC_Robin, cell_else::Array{BC_Robin,1}, Q_vent::Array{Float64,1})
    return sum( [ Q_vent[i] * (1005.0+1846.0*ah(cell_else[i])) * (353.25/temp(cell_else[i])) * temp(cell_else[i]) for i = 1:length(cell_else) ] )
end

function cal_c2_source(Hi::Float64)
    return Hi
end

cal_c2_source (generic function with 1 method)

### 6.1.3 計算式

In [60]:
# 多孔質材料中
cal_newtemp_by_ODE(cell_mns::Cell,     cell::Cell, cell_pls::Cell,     nx::Float64, dt::Float64 ) = cal_newtemp_by_ODE(temp(cell), cal_A1(cell), cal_c1x(cell_mns, cell, cell_pls), cal_c2x(cell_mns, cell, cell_pls), dt )
cal_newtemp_by_ODE(cell_mns::BC_Robin, cell::Cell, cell_pls::Cell,     nx::Float64, dt::Float64 ) = cal_newtemp_by_ODE(temp(cell), cal_A1(cell), cal_c1x(cell_mns, cell, cell_pls), cal_c2x(cell_mns, cell, cell_pls), dt )
cal_newtemp_by_ODE(cell_mns::Cell,     cell::Cell, cell_pls::BC_Robin, nx::Float64, dt::Float64 ) = cal_newtemp_by_ODE(temp(cell), cal_A1(cell), cal_c1x(cell_mns, cell, cell_pls), cal_c2x(cell_mns, cell, cell_pls), dt )
cal_newtemp_by_ODE(cell_mns::BC_Robin, cell::Cell, cell_pls::BC_Robin, nx::Float64, dt::Float64 ) = cal_newtemp_by_ODE(temp(cell), cal_A1(cell), cal_c1x(cell_mns, cell, cell_pls), cal_c2x(cell_mns, cell, cell_pls), dt )

# 空気中
function cal_newtemp_by_ODE(cell::BC_Robin, cell_trans::Array{Cell,1}, cell_vent::Array{BC_Robin,1}, Q_vent::Array{Float64,1}, Hi::Float64, dt::Float64)
     return cal_newtemp_by_ODE( temp(cell), cal_A1(cell), 
            cal_c1(cell, cell_trans, cell_vent, Q_vent), 
            cal_c2(cell, cell_trans, cell_vent, Q_vent, Hi), dt )
end

cal_newtemp_by_ODE (generic function with 7 methods)

## 6.2 水分収支式
### 6.2.1 基礎方程式  

水分収支式はその一般系として以下のように記述することができる。  

$$ A_2 \frac{\partial \mu_{(x,y,z)}}{\partial t} = c_3 \mu_{(x,y,z)} + c_4 $$ 

このとき、隣接する点の温度・湿度を一定値とみなせば、常微分方程式として解くことができる。  
常微分方程式の解は以下のように与えられる。  

$$ \mu_{(x,y,z)}^{t+\Delta t} = \Bigl( \mu_{(x,y,z)}^t + \frac{c_4}{c_3} \Bigl) exp \Bigl( \frac{c_3}{A_2}\Delta t \Bigl) - \frac{c_4}{c_3} $$ 


In [61]:
function cal_newmiu_by_ODE(miu::Float64, A2::Float64, c3::Float64, c4::Float64, dt::Float64 )
    return ( miu + c4 / c3 ) * exp( c3 / A2*dt ) - c4 / c3
end
cal_newmiu_by_ODE(;miu::Float64, A2::Float64, c3::Float64, c4::Float64, dt::Float64 ) = cal_newmiu_by_ODE(miu, A2, c3, c4, dt)

cal_newmiu_by_ODE (generic function with 6 methods)

### 6.2.2 各種係数の与え方

#### 6.2.2.1 $A_2$の与え方  
##### a) 多孔質材料中

In [62]:
cal_A2(rhow::Float64, V::Float64, dphi::Float64) = rhow * V * dphi
cal_A2(cell::Cell) = 998.0 * vol(cell) * dphi(cell)

cal_A2 (generic function with 2 methods)

##### b) 空気中

In [63]:
cal_A2(cell::Air) = ( 353.25/temp(cell) ) * vol(cell)
cal_A2(cell::BC_Robin) = ( 353.25/temp(cell) ) * vol(cell)

cal_A2 (generic function with 3 methods)

#### 6.2.2.2 $c_3$の与え方
##### a) 多孔質材料中
$c_3$は以下のように与えられる。

$$ c_3 = \sum_{i = x,y,z} \Bigl ( c_{3(i)}^{+} + c_{3(i)}^{-} \Bigl ) $$  


ただし、

$$ c_{3(x)} = - \Bigl( \frac{\lambda_{\mu(x)}^{'}}{dx2_{(x)}} + \frac{\lambda_{\mu(x-1)}^{'}}{dx2_{(x-1)}} \Bigl) $$
$$ c_{3(x)}^{-} = - \frac{\lambda_{\mu(x-1)}^{'}}{dx2_{(x-1)}} $$  
$$ c_{3(x)}^{+} = - \frac{\lambda_{\mu(x)}^{'}}{dx2_{(x)}}  $$  

In [64]:
function cal_c3x(cell_mns::Cell, cell::Cell, cell_pls::Cell)
    return cal_c3x_mns(cell, cell_mns) + cal_c3x_pls(cell, cell_pls)
end

function cal_c3x_mns(cell::Cell, cell_mns::Cell)
    dldmg_mns  = 1.0 / ( dldmgx_mns(cell) + dldmgx_pls(cell_mns)  )
    dldml_mns  = 1.0 / ( dldmlx_mns(cell) + dldmlx_pls(cell_mns)  )
    return - ( dldmg_mns + dldml_mns )
end

function cal_c3x_pls(cell::Cell, cell_pls::Cell)
    dldmg_pls  = 1.0 / ( dldmgx_pls(cell) + dldmgx_mns(cell_pls)  )
    dldml_pls  = 1.0 / ( dldmlx_pls(cell) + dldmlx_mns(cell_pls)  )
    return - ( dldmg_pls + dldml_pls )
end

cal_c3x_pls (generic function with 2 methods)

##### b) 空気境界

$$ c_{3(x)}^{-} = - \alpha_{\mu}^{'}$$  
$$ c_{3(x)}^{+} = - \alpha_{\mu}^{'} $$  

In [65]:
cal_c3x(cell_mns::BC_Robin, cell::Cell, cell_pls::Cell)     = cal_c3x_mns(cell, cell_mns) + cal_c3x_pls(cell, cell_pls)
cal_c3x(cell_mns::Cell,     cell::Cell, cell_pls::BC_Robin) = cal_c3x_mns(cell, cell_mns) + cal_c3x_pls(cell, cell_pls)
cal_c3x(cell_mns::BC_Robin, cell::Cell, cell_pls::BC_Robin) = cal_c3x_mns(cell, cell_mns) + cal_c3x_pls(cell, cell_pls)

function cal_c3x_mns(cell::Cell, cell_mns::BC_Robin)
    return - aldmu(cell_mns)
end

function cal_c3x_pls(cell::Cell, cell_pls::BC_Robin)
    return - aldmu(cell_pls)
end

cal_c3x_pls (generic function with 2 methods)

##### c) 空気中  

$$ c_3 = -\Sigma_{k=1}^{m}S_k \alpha_{X}^{'} -\Sigma_{j=1}^{n} Q_{ij} \rho $$

In [66]:
# メイン
function cal_c3(cell::BC_Robin, cell_trans::Array{Cell,1}, cell_vent::Array{BC_Robin,1}, Q_vent::Array{Float64,1})
    return cal_c3_trans(cell, cell_trans) + cal_c3_vent( cell, cell_vent, Q_vent)
end

# 伝達成分
function cal_c3_trans(cell::BC_Robin, cell_else::Array{Cell,1})
    return - sum( [ aldm(cell)/cal_dah_dpv(pv(cell), patm(cell) ) for i = 1:length(cell_else) ] )
end

# 換気成分
function cal_c3_vent( cell::BC_Robin, cell_else::Array{BC_Robin,1}, Q_vent::Array{Float64,1})
    return - sum( [ Q_vent[i] * (353.25/temp(cell)) for i = 1:length(cell_else) ] )
end

cal_c3_vent (generic function with 1 method)

#### 6.2.2.3 $c_4$の与え方
##### a) 多孔質材料中
$c_4$は以下のように与えられる。

$$ c_4 = \sum_{i = x,y,z} \Bigl ( c_{4(i)}^{+} + c_{4(i)}^{-} \Bigl ) $$  

ただし、
$$c_{4(x)} = \Bigl( \frac{\lambda_{\mu(x)}'}{dx2_{(x)}} \mu_{(x+1)} + \frac{\lambda_{\mu(x-1)}'}{dx2_{(x-1)}} \mu_{(x-1)} \Bigl) + \Bigl( \frac{\lambda_{T(x)}'}{dx2_{(x)}} (T_{(x+1)}-T_{(x)}) + \frac{\lambda_{T(x-1)}'}{dx2_{(x-1)}} (T_{(x)}-T_{(x-1)}) \Bigl) -\Bigl( - \frac{\lambda_{\mu(x)}'}{dx2_{(x)}} + \frac{\lambda_{\mu(x-1)}'}{dx2_{(x-1)}} \Bigl)n_x g $$
$$ c_{4(x)}^{-} = \frac{\lambda_{\mu(l-1)}'}{dx2_{(x-1)}} ( \mu_{(x-1)} - n_x g ) + \frac{\lambda_{T(x-1)}'}{dx2_{(x-1)}} (T_{(x)}-T_{(x-1)})$$  
$$ c_{4(x)}^{+} = \frac{\lambda_{\mu(l)}'}{dx2_{(x)}} ( \mu_{(x+1)} + n_x g ) + \frac{\lambda_{T(x)}'}{dx2_{(x)}} (T_{(x+1)}-T_{(x)})$$  

In [67]:
function cal_c4x(cell_mns::Cell, cell::Cell, cell_pls::Cell, nx::Float64)
    return cal_c4x_mns(cell, cell_mns, nx) + cal_c4x_pls(cell, cell_pls, nx)
end

function cal_c4x_mns(cell::Cell, cell_mns::Cell, nx::Float64 )
    dldmg_mns  = 1.0 / ( dldmgx_mns(cell) + dldmgx_pls(cell_mns)  )
    dldml_mns  = 1.0 / ( dldmlx_mns(cell) + dldmlx_pls(cell_mns)  )
    dldtg_mns  = 1.0 / ( dldtgx_mns(cell) + dldtgx_pls(cell_mns)  )
    return ( dldmg_mns + dldml_mns ) * miu(cell_mns) - dldml_mns * nx * 9.806650 + dldtg_mns * ( temp(cell) - temp(cell_mns) )
end

function cal_c4x_pls(cell::Cell, cell_pls::Cell, nx::Float64 )
    dldmg_pls  = 1.0 / ( dldmgx_pls(cell) + dldmgx_mns(cell_pls)  )
    dldml_pls  = 1.0 / ( dldmlx_pls(cell) + dldmlx_mns(cell_pls)  )
    dldtg_pls  = 1.0 / ( dldtgx_pls(cell) + dldtgx_mns(cell_pls)  )
    return ( dldmg_pls + dldml_pls ) * miu(cell_pls) + dldml_pls * nx * 9.806650 + dldtg_pls * ( temp(cell_pls) - temp(cell) )
end

cal_c4x_pls (generic function with 2 methods)

##### b)空気境界

$$ c_{4(x)}^{-} = \alpha_{\mu}{'} \mu_{(x-1)} + \alpha_{T}^{'}(T_{(x)}-T_{(x-1)})$$  
$$ c_{4(x)}^{+} = \alpha_{\mu}{'} \mu_{(x+1)} + \alpha_{T}^{'}(T_{(x+1)}-T_{(x)}) $$  

In [68]:
cal_c4x(cell_mns::BC_Robin, cell::Cell, cell_pls::Cell,     nx::Float64 ) = cal_c4x_mns(cell, cell_mns, nx) + cal_c4x_pls(cell, cell_pls, nx)
cal_c4x(cell_mns::Cell,     cell::Cell, cell_pls::BC_Robin, nx::Float64 ) = cal_c4x_mns(cell, cell_mns, nx) + cal_c4x_pls(cell, cell_pls, nx)
cal_c4x(cell_mns::BC_Robin, cell::Cell, cell_pls::BC_Robin, nx::Float64 ) = cal_c4x_mns(cell, cell_mns, nx) + cal_c4x_pls(cell, cell_pls, nx)

function cal_c4x_mns(cell::Cell, cell_mns::BC_Robin, nx::Float64 )
    return aldmu(cell_mns) * miu(cell_mns) + aldt(cell_mns) * ( temp(cell) - temp(cell_mns) )
end

function cal_c4x_pls(cell::Cell, cell_pls::BC_Robin, nx::Float64 )
    return aldmu(cell_pls) * miu(cell_pls) + aldt(cell_pls) * ( temp(cell) - temp(cell_pls) )
end

cal_c4x_pls (generic function with 2 methods)

##### c) 空気中  

$$ c_4 = \Sigma_{k=1}^{m}S_k \alpha_{X}^{'} X_k + \Sigma_{j=1}^{n} Q_{ij} \rho X_k + H^{'} $$

In [69]:
# メイン
function cal_c4(cell::BC_Robin, cell_trans::Array{Cell,1}, cell_vent::Array{BC_Robin,1}, Q_vent::Array{Float64,1}, Hi::Float64)
    return cal_c4_trans(cell, cell_trans) + cal_c4_vent( cell, cell_vent, Q_vent) + cal_c4_source(Hi)
end

# 伝達成分
function cal_c4_trans(cell::BC_Robin, cell_else::Array{Cell,1})
    return sum( [ aldm(cell)/cal_dah_dpv(pv(cell), patm(cell) ) * ah(cell_else[i]) for i = 1:length(cell_else) ] )
end

# 換気成分
function cal_c4_vent( cell::BC_Robin, cell_else::Array{BC_Robin,1}, Q_vent::Array{Float64,1})
    return sum( [ Q_vent[i] * (353.25/temp(cell_else[i])) * ah(cell_else[i]) for i = 1:length(cell_else) ] )
end

function cal_c4_source(Hi::Float64)
    return Hi
end

cal_c4_source (generic function with 1 method)

### 6.2.3 計算式

In [70]:
# 多孔質材料中
cal_newmiu_by_ODE(cell_mns::Cell,     cell::Cell, cell_pls::Cell,     nx::Float64, dt::Float64 )  = cal_newmiu_by_ODE(miu(cell), cal_A2(cell), cal_c3x(cell_mns, cell, cell_pls), cal_c4x(cell_mns, cell, cell_pls, nx), dt )
cal_newmiu_by_ODE(cell_mns::BC_Robin, cell::Cell, cell_pls::Cell,     nx::Float64, dt::Float64 )  = cal_newmiu_by_ODE(miu(cell), cal_A2(cell), cal_c3x(cell_mns, cell, cell_pls), cal_c4x(cell_mns, cell, cell_pls, nx), dt )
cal_newmiu_by_ODE(cell_mns::Cell,     cell::Cell, cell_pls::BC_Robin, nx::Float64, dt::Float64 )  = cal_newmiu_by_ODE(miu(cell), cal_A2(cell), cal_c3x(cell_mns, cell, cell_pls), cal_c4x(cell_mns, cell, cell_pls, nx), dt )
cal_newmiu_by_ODE(cell_mns::BC_Robin, cell::Cell, cell_pls::BC_Robin, nx::Float64, dt::Float64 )  = cal_newmiu_by_ODE(miu(cell), cal_A2(cell), cal_c3x(cell_mns, cell, cell_pls), cal_c4x(cell_mns, cell, cell_pls, nx), dt )

# 空気中
function cal_newAH_by_ODE(cell::BC_Robin, cell_trans::Array{Cell,1}, cell_vent::Array{BC_Robin,1}, Q_vent::Array{Float64,1}, Hi::Float64, dt::Float64)
    return cal_newmiu_by_ODE( ah(cell), cal_A2(cell), 
           cal_c3(cell, cell_trans, cell_vent, Q_vent), 
           cal_c4(cell, cell_trans, cell_vent, Q_vent, Hi), dt )
end

cal_newAH_by_ODE (generic function with 1 method)

## 検証

In [71]:
using Dates # 解析内時間を計算するためのモジュール
include("../module/air.jl");  # 空間の構成要素を表すモジュール
include("../module/boundary_condition.jl"); # 境界条件を表すモジュール
include("../module/transfer_in_media.jl"); # 材料中における熱・水分の移動を表すモジュール
include("../module/climate_data.jl"); # 気象データを取り扱うモジュール
include("../logger.jl"); # 計算結果を保存するモジュール
target_model = input_cell_data("../input_data/1D_model/WG_benchmark_stage0.csv")

56-element Vector{Any}:
 BC_Robin("NoName", Air("air_in", 1, 0.0, 0.0, 0.0, 1.0, 293.15, 0.45, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), Cell([1, 1, 1], [0.0, 0.0, 0.0], 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 101325.0, "NoName", [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]), 0.0, 0.0, 0.0, 9.3, 4.4, 4.9, 2.90797836530015e-8, "E", 0.0, 0.0)
 Cell([2, 1, 1], [0.0, 0.0, 0.0], 0.001, 1.0, 1.0, 0.0, 1.0, 1.0, 293.15, -108120.1202194354, 0.0, 0.0, 0.0, 101325.0, "plasterboard", [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
 Cell([3, 1, 1], [0.0, 0.0, 0.0], 0.001, 1.0, 1.0, 0.0005, 1.0, 1.0, 293.15, -108120.1202194354, 0.0, 0.0, 0.0, 101325.0, "plasterboard", [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
 Cell([4, 1, 1], [0.0, 0.0, 0.0], 0.001, 1.0, 1.0, 0.0005, 1.0, 1.0, 293.15, -108120.1202194

In [73]:
target_model[1].air.temp = 300.15
target_model[1].air.rh = 0.4
target_model[end].air.temp = 300.15
target_model[end].air.rh = 0.4
target_model[4].temp = 300.15
target_model[4].miu  = -10000.0

-10000.0

In [74]:
print(temp(target_model[2]),"   ", miu(target_model[2]))

293.15   -108120.1202194354

In [75]:
cal_c1x_mns( target_model[2], target_model[1]) / cal_c2x_mns(target_model[2], target_model[1])

-0.003341587760073147

In [76]:
cal_c2x_pls(target_model[2], target_model[3]) / cal_c1x_pls( target_model[2], target_model[3])

-293.15

In [77]:
cal_A1(target_model[2])
cal_A2(target_model[2])
cal_c1x(target_model[1], target_model[2], target_model[3])
cal_c2x(target_model[1], target_model[2], target_model[3])
cal_c3x(target_model[1], target_model[2], target_model[3])
cal_c4x(target_model[1], target_model[2], target_model[3], 0.0)
println(cal_newtemp_by_ODE(target_model[1], target_model[2], target_model[3], 0.0, 10.0))
println(cal_newmiu_by_ODE(target_model[1], target_model[2], target_model[3], 0.0, 10.0))

293.66849007328113
-108120.12045544012


In [78]:
println(cal_newtemp_by_ODE(target_model[2], target_model[3], target_model[4], 0.0, 10.0))
println(cal_newmiu_by_ODE(target_model[2], target_model[3], target_model[4], 0.0, 1000.0))

297.5160549418253
-108120.07667439966


In [79]:
cal_A1(target_model[end-1])
cal_A2(target_model[end-1])
cal_c1x(target_model[end-2], target_model[end-1], target_model[end])
cal_c2x(target_model[end-2], target_model[end-1], target_model[end])
cal_c3x(target_model[end-2], target_model[end-1], target_model[end])
cal_c4x(target_model[end-2], target_model[end-1], target_model[end], 0.0)

-5.711849491824129e-5

In [80]:
println(cal_newtemp_by_ODE(target_model[end-2], target_model[end-1], target_model[end], 0.0, 10.0))
println(cal_newmiu_by_ODE(target_model[end-2], target_model[end-1], target_model[end], 0.0, 10.0))

293.62633211631027
-108120.12037677188
